In [4]:
print('lets get these things imported')
import pandas as pd
import numpy as np
import matplotlib 


import category_encoders as ce
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

# we probably won't need these - for now
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# adding some more in here
# i don't think we want nltk. 
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn import cluster
#

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import GradientBoostingClassifier


from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import plot_tree


lets get these things imported


In [5]:
dfreal = pd.read_csv('True.csv',
                    parse_dates = ['date'])
                    #index_col = 'date')
dfreal['Fake'] = 0
dfreal.shape


(21417, 5)

In [6]:
dfreal.head()

,title,text,subject,date,Fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,2017-12-31,0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,2017-12-29,0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,2017-12-31,0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,2017-12-30,0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,2017-12-29,0


In [7]:
dfreal_trimmed = dfreal[-21_400 :]
dfreal_trimmed.shape

(21400, 5)

In [8]:

dffake = pd.read_csv('Fake.csv',
                    parse_dates = ['date'])
                    #index_col = 'date')

dffake['Fake'] = 1
dffake.shape

(23481, 5)

In [9]:
dffake.head()

,title,text,subject,date,Fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [10]:
# Alright, we should be happy with that. Shame it took so long to get there. 
# We are adding a boolean column for "Fake" (0 will mean real news)


In [11]:
dffake_trimmed = dffake[-21_400 :]
print(dffake_trimmed.shape)

(21400, 5)


In [12]:
# frames = [dfreal_trimmed, dffake_trimmed]
# df_joined = pd.concat(frames)
# df_joined.shape
df_joined = dfreal_trimmed.append(dffake_trimmed, ignore_index=True)
# Trimming everything up to be right around (42_000, 5) or so merged length with  
df_joined.shape

(42800, 5)

In [13]:
df_joined.head()

,title,text,subject,date,Fake
0,"Trump on Twitter (Dec 26) - Hillary Clinton, T...",The following statements were posted to the ve...,politicsNews,2017-12-26 00:00:00,0
1,U.S. appeals court rejects challenge to Trump ...,(Reuters) - A U.S. appeals court in Washington...,politicsNews,2017-12-26 00:00:00,0
2,Treasury Secretary Mnuchin was sent gift-wrapp...,(Reuters) - A gift-wrapped package addressed t...,politicsNews,2017-12-24 00:00:00,0
3,Federal judge partially lifts Trump's latest r...,WASHINGTON (Reuters) - A federal judge in Seat...,politicsNews,2017-12-24 00:00:00,0
4,Exclusive: U.S. memo weakens guidelines for pr...,NEW YORK (Reuters) - The U.S. Justice Departme...,politicsNews,2017-12-23 00:00:00,0


In [14]:
df_joined.describe()

,Fake
count,42800.000000
mean,0.500000
std,0.500006
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [15]:
# Now I need to pass this through the NLP functions. Let's get those set up... 

# first a single row just to test... 

df_test01 = df_joined[: 2]
df_test01.head()

,title,text,subject,date,Fake
0,"Trump on Twitter (Dec 26) - Hillary Clinton, T...",The following statements were posted to the ve...,politicsNews,2017-12-26 00:00:00,0
1,U.S. appeals court rejects challenge to Trump ...,(Reuters) - A U.S. appeals court in Washington...,politicsNews,2017-12-26 00:00:00,0


# Count Vectorizer

In [16]:
from pandas import DataFrame

cv = CountVectorizer(stop_words = 'english', strip_accents ='ascii', max_features = 50, max_df = 0.95 , min_df = 0.01 )
# test01_trans = cv.fit_transform(df_test01).toarray()
# dftest_cv = pd.DataFrame(test01_trans,
#                         columns=cv.get_feature_names())

#change index labels - ask why
#dftest_cv.index = [f'doc_{n}' for n in dftest_cv.index]

def create_term_matrix(message_list, vectorizer):
    doc_term_df = vectorizer.fit_transform(message_list)
    return DataFrame(doc_term_df.toarray(),
                     columns=vectorizer.get_feature_names())
                    
#create_term_matrix(df_test01['text'], cv)



df_fullpass_1000_025to005 = create_term_matrix(df_joined['text'], cv)


# create a dictionary for myself (ex: year and years same word)
# find an NLP function that can cluster similiar words together - lookup common NLP functions. 

In [22]:
df_fullpass_1000_025to005.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42800 entries, 0 to 42799
Data columns (total 50 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   according     42800 non-null  int64
 1   american      42800 non-null  int64
 2   called        42800 non-null  int64
 3   campaign      42800 non-null  int64
 4   clinton       42800 non-null  int64
 5   country       42800 non-null  int64
 6   court         42800 non-null  int64
 7   democratic    42800 non-null  int64
 8   did           42800 non-null  int64
 9   donald        42800 non-null  int64
 10  election      42800 non-null  int64
 11  going         42800 non-null  int64
 12  government    42800 non-null  int64
 13  hillary       42800 non-null  int64
 14  house         42800 non-null  int64
 15  just          42800 non-null  int64
 16  law           42800 non-null  int64
 17  like          42800 non-null  int64
 18  make          42800 non-null  int64
 19  media         42800 non-n

In [18]:
df_fullpass_1000_025to005.describe()

,according,american,called,campaign,clinton,country,court,democratic,did,donald,...,support,time,told,trump,united,washington,week,white,year,years
count,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,...,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000
mean,0.314860,0.360047,0.295864,0.483551,0.617967,0.389182,0.329229,0.303621,0.369579,0.589720,...,0.297453,0.460584,0.529463,2.516986,0.536192,0.421916,0.299743,0.482874,0.557079,0.386706
std,0.787661,1.064934,0.659172,1.348366,2.304393,0.904831,1.352565,0.888218,0.797904,1.019764,...,0.767365,0.909996,0.939871,4.381324,1.283519,0.954844,0.675265,1.305999,1.120036,0.890860
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,...,0.000000,1.000000,1.000000,3.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,20.000000,48.000000,10.000000,31.000000,67.000000,26.000000,43.000000,18.000000,20.000000,32.000000,...,24.000000,28.000000,13.000000,73.000000,42.000000,35.000000,10.000000,36.000000,27.000000,61.000000


In [23]:
df_fullpass_1000_025to005.head()

,according,american,called,campaign,clinton,country,court,democratic,did,donald,...,support,time,told,trump,united,washington,week,white,year,years
0,0,0,0,2,1,0,0,0,0,1,...,0,2,0,3,0,0,0,0,3,0
1,0,0,0,0,1,0,3,3,1,1,...,0,0,0,1,1,2,0,0,0,0
2,1,0,0,0,0,0,0,0,0,1,...,0,0,3,2,0,0,1,0,0,0
3,1,0,2,0,0,2,3,0,0,1,...,0,0,0,5,3,1,0,0,1,1
4,0,0,1,0,0,1,2,0,0,1,...,0,3,0,2,1,0,0,0,1,0


# next we try for - TFIDF Vectorizer -
# remove stopwords - essential - do not forget - also can add my own stop words...
#   - so, run it once then use intuition to see what other words should not be in there.
# lemmatization vs clustering - find out where it is in these packages. 

In [20]:
# # Instantiate vectorizer
# tfidf = TfidfVectorizer(stop_words = 'english')

# # Vectorize bad_revs
# test_02_trans = tfidf.fit_transform(df_test01['text']).toarray()

# # Create DataFrame for display below
# dftest_tfidf = pd.DataFrame(np.round(test_02_trans, 2),
#                         columns=tfidf.get_feature_names())

# # Change index labels
# dftest_tfidf.index = [f'doc_{n}' for n in dftest_tfidf.index]

# # Show result
# dftest_tfidf

# and now we can try out Hashing Vectorizer

In [21]:
# # Instantiate vectorizer
# hv = HashingVectorizer(alternate_sign=False, norm=None, stop_words = 'english')

# # Vectorize bad_revs
# test_03_trans_hv = hv.transform(df_test01).toarray()

# # Create DataFrame for display below
# df_test_hv = pd.DataFrame(test_03_trans_hv)

# # Change index labels
# df_test_hv.index = [f'doc_{n}' for n in df_test_hv.index]

# # Show result
# df_test_hv

# I'm ready to to build a pipeline out for a random forest model but these results don't really seem actionable. I spent too much of today on the VSCode setup, but generally speaking it was all worth it. 

# obviously the target here will be "Fake" - the question is how to split it. 
# let's remember that we imported these new things also...
# from nltk.stem import PorterStemmer
# from nltk.corpus import stopwords
# from sklearn import cluster
#

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import HashingVectorizer 